Updates to script for version 2

1.   Using PySpark & Spark Pandas API for better performance & speed
2.   Using Azure Cousmos DB to send cleaned data to PowerBI Dashboard
3.   Minor usage of Numba Framework to speed up some methods within this script
4.   Potential Trials of Time-Series Machine Learning Modeling of Data 
5.   Trials with using the Libra Library for potential simple Machine Learning development
6.   Determine if using Azure Synapse makes sense for this project for giving no-code manipulation of the data

Website Links & Documentation for all program enhancements

1.   PySpark Examples - https://github.com/apache/spark/tree/f74867bddf/examples/src/main/python
2.   Azure Cosmos DB - https://docs.microsoft.com/en-us/azure/cosmos-db/sql/create-sql-api-python
2.   Azure Cosmos DB Github Demo - https://github.com/Azure-Samples/azure-cosmos-db-python-getting-started/blob/main/cosmos_get_started.py
3.   Numba - https://numba.pydata.org/
4.   Time-Series Machine Learning Algos - https://www.advancinganalytics.co.uk/blog/2021/06/22/10-incredibly-useful-time-series-forecasting-algorithms
5.   Libra - https://libradocs.org/
6.   Azure Synapse & Cosmos DB Integration - https://docs.microsoft.com/en-us/azure/cosmos-db/synapse-link
7.   Spark SQL & Python - https://spark.apache.org/docs/latest/sql-programming-guide.html
8.   DeepHaven Streaming Dataframe - https://deephaven.io/?utm_term=streaming%20data&utm_campaign=Website+Traffic+Q4+2022&utm_source=adwords&utm_medium=ppc&hsa_acc=4673439537&hsa_cam=18322365595&hsa_grp=139855244374&hsa_ad=621564846172&hsa_src=g&hsa_tgt=kwd-161093182&hsa_kw=streaming%20data&hsa_mt=p&hsa_net=adwords&hsa_ver=3
9.   Spark SQL & Python - https://spark.apache.org/docs/latest/
10.  Google Sheets & Python - https://ploomber.io/blog/gsheets/

In [3]:
#all imports 
import gspread
import numpy as np
import pyspark
import pyspark.pandas as ps
import numba
from numba import jit
import os
import json
from datetime import *

In [4]:
#setup spark env 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/18 22:49:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#Import the data from google sheets
"""
sa = gspread.service_account(filename="/com.docker.devenvironments.code/kuthesisdataclean-558333f9362c.json")
sheet = sa.open("POU_Unclean_Data")
work_sheet = sheet.worksheet("POU")
rows = work_sheet.get_all_values()
pou_data = ps.DataFrame.from_records(rows)
spark_df = spark.createDataFrame(rows).get_all_records()
"""


AttributeError: 'DataFrame' object has no attribute 'get_all_records'

In [6]:
#call to the dataset within Google Sheets
sa = gspread.service_account(filename="/com.docker.devenvironments.code/kuthesisdataclean-558333f9362c.json")

In [10]:
#Create a method that return the values from the google sheet
@jit
def pull_df(sa):
    sheet = sa.open("POU_Unclean_Data")
    work_sheet = sheet.worksheet("POU")
    df = ps.Dataframe(work_sheet.get_all_records())
    spark_df = spark.createDataFrame(df)

In [11]:
spark_df = pull_df(sa)
#spark_df = pull_df(spark_df)
#spark_df

/tmp/ipykernel_2393/4249707886.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "pull_df" failed type inference due to: Untyped global name 'spark': Cannot determine Numba type of <class 'pyspark.sql.session.SparkSession'>

File "tmp/ipykernel_2393/4249707886.py", line 7:
<source missing, REPL/exec in use?>

  @jit
/home/vscode/micromamba/envs/py39/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "pull_df" was compiled in object mode without forceobj=True.

File "tmp/ipykernel_2393/4249707886.py", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/vscode/micromamba/envs/py39/lib/python3.10/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedo

AttributeError: module 'pyspark.pandas' has no attribute 'Dataframe'

In [ ]:
#Create a new spark dataframe to use for analysis
@jit
def create_df():
    

In [9]:
#Create Spark Dataframe with the data within the Google Sheet
#spark_df = spark.createDataFrame(pou_data).toDF(columns_*)
#val dfFromRDD2 = spark.createDataFrame(rdd).toDF(columns:_*)
#spark_df.head()

ModuleNotFoundError: No module named 'spark'

In [ ]:
#Format the dataframe so we can start working on using it for data analysis
#Next we are going to format the rows to allow for the data to get into a data frame
spark_df  = spark_df.drop(labels=0, axis=0)
spark_df  = spark_df.reset_index(drop=True)
spark_df.columns = spark_df.loc[0]
spark_df  = spark_df.loc[1:].reset_index(drop=True)
spark_df.drop_duplicates(subset="Serial Number", keep='last')
spark_df

In [18]:
@jit
def format_df(df):
    df = df.drop(labels=0, axis=0)
    df = df.index(drop=True)
    df.columns = df.collect()[0]
    df = df[1:].collect(drop=True)
    df.drop_duplicates(subset="Serial Number", keeps='last')

In [20]:
spark_df = format_df(spark_df)

NameError: name 'spark_df' is not defined